In [7]:
# kafka.motorflowneo.iii-ei-stack.com:32510
# kafka.motorflowneo.iii-ei-stack.com:32511
# kafka.motorflowneo.iii-ei-stack.com:32512

In [20]:
from kafka import KafkaProducer
from os import listdir
from os.path import isfile, join

import pandas as pd
import time
import json

In [21]:
load_path = '../datasets/phm-2012/Learning_set/Bearing1_1/'
onlyfiles = [f for f in listdir(load_path) if isfile(join(load_path, f))]
onlyfiles.sort()

In [22]:
def producer(x, y):
    
    producer = KafkaProducer(bootstrap_servers=['59.124.112.101:32110'])
    
    data = json.dumps({
        "x" : x,
        "y": y
    })
    x = bytes(str(x), encoding = "utf8")
    y = bytes(str(y), encoding = "utf8")
    data_encoded = bytes(data, encoding = "utf8")
    
    future = producer.send('phm-x' , key= b'horizontal', value= x, partition= 0)
    result = future.get(timeout= 1000)
    
    future = producer.send('phm-y' , key= b'vertical', value= y, partition= 0)
    result = future.get(timeout= 1000)
    
    future = producer.send('phm-mix' , key= b'mixed', value= data_encoded, partition= 0)
    result = future.get(timeout= 1000)
    
#     print(result)

In [ ]:
while (True):

    for file in onlyfiles:
        
        #times in seconds
        ts = int(file.split('_')[1].split('.')[0])
        
        payload_x=[ts]
        payload_y=[ts]

        producer_pd = pd.read_csv(load_path+file, header=None)

        anchor_head=2560
        length=2560

        for anchor in range(0, int(len(producer_pd)/length)):

            horizontal = producer_pd.head(anchor_head).tail(length)[[4]].T.values.tolist()[0]
            vertical = producer_pd.head(anchor_head).tail(length)[[5]].T.values.tolist()[0]
            
            payload_x = payload_x + horizontal
            payload_y = payload_y + vertical
            
            producer(payload_x, payload_y)
            
            # calculas timestamp for each 512 datapoints, total datapoints = 2803 * 5 = 14015 seconds
#             ts = (int(file.split('_')[1].split('.')[0])-1) * int(len(producer_pd)/length) + int(anchor_head/length)
#             print(ts)
            
            anchor_head=anchor_head + length

            time.sleep(10)

In [ ]:
# producer_pd

In [ ]:
file = 'acc_00001.csv'